# Gazete Tiraj Verileri
Bu script, haftalık gazete satış rakamlarını çekmektedir. Aşağıda verilen kaynakta haftalık olarak en çok satana göre 20 gazete sıralanmaktadır.

**Senaryo**: Sayfada veri ajax ile güncellenmektedir. Sitenin sağ üst köşesinde yer alan tarih seçici ile hafta değiştirilir. Yıl içerisindeki her pazartesi seçilerek sayfada bulunan tablo güncellenir. Güncellemeden sonra sayfadaki tablo çekilerek DataFrame'e dönüştürülür ve bir listeye eklenir. Bir sonraki Pazartesi'ye geçilir.

**Kapsam**: Görünüşe göre web sitesinde 2015'e kadar veri bulunmakta. 2015-2019 aralığındaki veriler yıllık olarak çekilmiştir.

**Başlıklar**

*   date (Tarih)
*   newspaper (Gazete)
*   circulation (Tiraj)

**Erişim tarihi**: 01.11.2019

**Kaynak**: gazetetirajları.com

**Not**: Gazete tirajları resmi bir kaynaktan yapısal bir şekilde paylaşılmadığından, en sağlıklı kaynak olarak bu web sitesi seçilmiştir. Barındırdığı satış rakamlarının güvenilirliği test edilememektedir.

Umut Irmaksever @ AVVG

Gerekli paketler yüklenir ve Selenium instance'ı başlatılır, web sayfasına gidilir.

In [15]:
# install chromium, its driver, and selenium
# !apt install chromium-chromedriver
# !pip install selenium html5lib bs4
# !pip install BeautifulSoup4
# !pip install html5lib
# !pip install matplotlib
# set options to be headless, ..
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By as By

import html5lib
import pandas as pd

import datetime
from time import sleep

options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--window-size=1920,1080")
# open it, go to a website, and get results
wd = webdriver.Chrome('./chromedriver',options=options)
page = wd.get("http://gazetetirajlari.com/HaftalikTirajlar.aspx")

In [20]:
searchbox = wd.find_element_by_id("txtHaftalikTirajTarih")
datatable = wd.find_elements_by_tag_name("table")[0].get_attribute("outerHTML")
datatable_pd = pd.read_html(datatable, flavor="bs4")
datatable_pd

[    Unnamed: 0  Unnamed: 1   GAZETE ADI   GÜNCEL  GEÇEN HAFTA  FARK +/-
 0            1         NaN     HÜRRİYET  195.309      194.146     1.164
 1            2         NaN        SABAH  193.987      193.833   154.000
 2            3         NaN        SÖZCÜ  186.436      184.961     1.475
 3            4         NaN        POSTA  124.634      122.432     2.202
 4            5         NaN     MİLLİYET  123.565      123.778  -213.000
 5            6         NaN      TÜRKİYE  123.508      123.414    94.000
 6            7         NaN   YENİ ŞAFAK  102.516      102.403   113.000
 7            8         NaN        AKŞAM  100.286      100.010   276.000
 8            9         NaN       TAKVİM   87.087       87.100   -12.000
 9           10         NaN    YENİ AKİT   56.230       56.343  -113.000
 10          11         NaN     KORKUSUZ   55.982       56.532  -550.000
 11          12         NaN        KARAR   51.304       51.300     4.000
 12          13         NaN  YENİ BİRLİK   50.251  

Bir yıl içerisindeki tüm pazartesileri bir listeye kaydedelim.

In [21]:
def allmondays(year):
    # Yılın ilk günü
    d = datetime.date(year, 1, 1)
    # Pazartesi'yi bul
    d += datetime.timedelta(days=7-d.weekday())
    # Yıl aynı olduğu sürece 7 gün artırarak devam et 
    while d.year == year:
        yield d
        d+= datetime.timedelta(days=7)
    return(d)
# Listedeki tüm tarih objelerini istenen formata sok-
def get_mondays_list(year):
    mondays = allmondays(year)
    mondays = list(map(lambda x: x.strftime("%d.%m.%Y"), mondays))
    return mondays


Sayfadaki veri tablosunu bulup DataFrame'e çevirelim.

In [22]:
def get_circulation_data():
    data_table_div = wd.find_element_by_id("divHaftalikTirajAra")
    week = data_table_div.find_elements_by_tag_name("h3")[0].text
    data_table = data_table_div.find_elements_by_tag_name("table")[0].get_attribute("outerHTML")
    data_table_pd = pd.read_html(data_table, flavor="bs4")
    data_table_pd = data_table_pd[0]
    return data_table_pd

Sayfadaki veri tablosunun üstünde başlık olarak o haftanın tarih aralığı belirtiliyor. Doğrulama amaçlı kullanılabilir. İkinci fonksiyon ise datepicker widget'ına istenen tarihi girip butona basıyor.

In [23]:
def get_current_week():
    week = wd.find_element_by_id("divHaftalikTirajAra").find_elements_by_tag_name("h3")[0].text
    return week
def change_date(date):
    date_textfield = wd.find_element_by_id("txtHaftalikTirajTarih")
    date_textfield.clear()
    # date_textfield.click()
    date_textfield.send_keys(date)
    wd.find_element_by_id("btnHaftalikTirajAra").click()
    # Tarih değiştirildikten sonra bir "Yükleniyor" mesajıyla bir modal çıkıyor.
    # Scriptin sekteye uğramaması için bu modal kapanana kadar bekle.
    element = WebDriverWait(wd, 10).until(
        EC.invisibility_of_element_located((By.ID, 'loading')))

Aşağıdaki helper (yardımcı) fonksiyon ile bir ya da daha fazla yıl içerisindeki Pazartesiler datepicker'da deneniyor ve o hafta için veri olup olmadığı belirleniyor. Veri bulunan haftalar "success_weeks" listesine, olmayan haftalar ise "fail_weeks" listesine kaydediliyor.

In [25]:
weeks_dict = {}
years = range(2020, 2021)
success_weeks = []
fail_weeks = []
def check_if_week_exists(years):
    for year in years:
        mondays_list = get_mondays_list(year)
        for week in mondays_list:
            change_date(week)
            try:
                week_from_website = get_current_week()
                print(week, week_from_website)
                # Hem python tarafından sağlanan tarih, hem de web sitesinden
                # alınan hafta stringini kaydet.
                success_weeks.append([week, week_from_website])
            except IndexError:
                print("nonexistent")
                fail_weeks.append([week, "fail"])
check_if_week_exists(years)

06.01.2020 06 - 12 OCAK 2020
13.01.2020 13 - 19 OCAK 2020
20.01.2020 20 - 26 OCAK 2020
27.01.2020 27 OCAK - 02 ŞUBAT 2020
03.02.2020 03 - 09 ŞUBAT 2020
10.02.2020 10 - 16 ŞUBAT 2020
17.02.2020 17 - 23 ŞUBAT 2020
24.02.2020 24 ŞUBAT - 01 MART 2020
02.03.2020 02 - 08 MART 2020
09.03.2020 09 - 15 MART 2020
16.03.2020 16 - 22 MART 2020
23.03.2020 23 - 29 MART 2020
30.03.2020 30 MART - 05 NISAN 2020
06.04.2020 06 - 12 NISAN 2020
13.04.2020 13 - 19 NISAN 2020
20.04.2020 20 - 26 NISAN 2020
27.04.2020 27 NISAN - 03 MAYIS 2020
04.05.2020 04 - 10 MAYIS 2020
11.05.2020 11 - 17 MAYIS 2020
nonexistent
25.05.2020 25 - 31 MAYIS 2020
01.06.2020 01 - 07 HAZIRAN 2020
08.06.2020 08 - 14 HAZIRAN 2020
15.06.2020 15 - 21 HAZIRAN 2020
22.06.2020 22 - 28 HAZIRAN 2020
29.06.2020 29 HAZIRAN - 05 TEMMUZ 2020
06.07.2020 06 - 12 TEMMUZ 2020
13.07.2020 13 - 19 TEMMUZ 2020
20.07.2020 20 - 26 TEMMUZ 2020
27.07.2020 27 TEMMUZ - 02 AĞUSTOS 2020
03.08.2020 03 - 09 AĞUSTOS 2020
10.08.2020 10 - 16 AĞUSTOS 2020
17.08.2020 

Veri olmayan haftaları kaydedelim.

In [27]:
# Fail olan haftaları bir dosyaya yazmak için aşağıdaki dosya ismini değiştirebilirsiniz.
with open("fails_2020.txt", mode="w+")as file:
    for fail in fail_weeks:
        file.write(str(fail) + "\n")
    file.close()

Her hafta için veri tablosunu bir dict objesine kaydedelim.

In [30]:
weekly_circulation_list = {}
for week in success_weeks:
    print(week)
    change_date(week[0])
    sleep(0.5)
    circulation_data = get_circulation_data()
    # print(circulation_data)
    weekly_circulation_list[week[0]] = circulation_data

['06.01.2020', '06 - 12 OCAK 2020']
['13.01.2020', '13 - 19 OCAK 2020']
['20.01.2020', '20 - 26 OCAK 2020']
['27.01.2020', '27 OCAK - 02 ŞUBAT 2020']
['03.02.2020', '03 - 09 ŞUBAT 2020']
['10.02.2020', '10 - 16 ŞUBAT 2020']
['17.02.2020', '17 - 23 ŞUBAT 2020']
['24.02.2020', '24 ŞUBAT - 01 MART 2020']
['02.03.2020', '02 - 08 MART 2020']
['09.03.2020', '09 - 15 MART 2020']
['16.03.2020', '16 - 22 MART 2020']
['23.03.2020', '23 - 29 MART 2020']
['30.03.2020', '30 MART - 05 NISAN 2020']
['06.04.2020', '06 - 12 NISAN 2020']
['13.04.2020', '13 - 19 NISAN 2020']
['20.04.2020', '20 - 26 NISAN 2020']
['27.04.2020', '27 NISAN - 03 MAYIS 2020']
['04.05.2020', '04 - 10 MAYIS 2020']
['11.05.2020', '11 - 17 MAYIS 2020']
['25.05.2020', '25 - 31 MAYIS 2020']
['01.06.2020', '01 - 07 HAZIRAN 2020']
['08.06.2020', '08 - 14 HAZIRAN 2020']
['15.06.2020', '15 - 21 HAZIRAN 2020']
['22.06.2020', '22 - 28 HAZIRAN 2020']
['29.06.2020', '29 HAZIRAN - 05 TEMMUZ 2020']
['06.07.2020', '06 - 12 TEMMUZ 2020']
['13.0

Bir yıldaki tüm veri olan haftaların bulunduğu dict objesindeki değerler DataFrame şeklinde. Web sayfasındaki tabloyu bire bir kaydettiğimiz için sadece ihtiyacımız olanları alalım ve DataFrame listesini düzleştirelim.

In [43]:
rows_list = []
for circulation_table in weekly_circulation_list.keys():
    week = circulation_table
    table = weekly_circulation_list[circulation_table]
    for row in table.index:
        newspaper_name = table["GAZETE ADI"][row]
        sales = table["GÜNCEL"][row]
        sales = int(sales * 1000)
        print(sales, type(sales))
        rows_list.append([week, newspaper_name, sales])
        print(week, newspaper_name, sales)
    print(circulation_table)

242274 <class 'int'>
06.01.2020 SABAH 242274
241577 <class 'int'>
06.01.2020 SÖZCÜ 241577
215630 <class 'int'>
06.01.2020 HÜRRİYET 215630
163059 <class 'int'>
06.01.2020 POSTA 163059
130877 <class 'int'>
06.01.2020 TÜRKİYE 130877
125499 <class 'int'>
06.01.2020 MİLLİYET 125499
111098 <class 'int'>
06.01.2020 AKŞAM 111098
104731 <class 'int'>
06.01.2020 TAKVİM 104731
102506 <class 'int'>
06.01.2020 YENİ ŞAFAK 102506
64855 <class 'int'>
06.01.2020 FANATİK 64855
63544 <class 'int'>
06.01.2020 P.FOTOMAÇ 63544
58972 <class 'int'>
06.01.2020 KORKUSUZ 58972
56436 <class 'int'>
06.01.2020 YENİ AKİT 56436
50974 <class 'int'>
06.01.2020 YENİ ASIR 50974
50623 <class 'int'>
06.01.2020 YENİ BİRLİK 50623
28384 <class 'int'>
06.01.2020 CUMHURİYET 28384
22358 <class 'int'>
06.01.2020 KARAR 22358
14226 <class 'int'>
06.01.2020 AYDINLIK 14226
13001 <class 'int'>
06.01.2020 DOĞRU HABER 13001
06.01.2020
242417 <class 'int'>
13.01.2020 SABAH 242417
241687 <class 'int'>
13.01.2020 SÖZCÜ 241687
216184 <class

Düzleştirilmiş listeyi tekrardan DataFrame'e çevirerek csv formatında kaydedelim.

In [44]:
rows_list_pd = pd.DataFrame(rows_list, columns=["date", "newspaper", "circulation"])
rows_list_pd.to_csv("tr_newspapers_2020_26.12.2020.csv")
rows_list_pd

,date,newspaper,circulation
0,06.01.2020,SABAH,242274
1,06.01.2020,SÖZCÜ,241577
2,06.01.2020,HÜRRİYET,215630
3,06.01.2020,POSTA,163059
4,06.01.2020,TÜRKİYE,130877
...,...,...,...
926,14.12.2020,FANATİK,43086
927,14.12.2020,P.FOTOMAÇ,37979
928,14.12.2020,CUMHURİYET,25191
929,14.12.2020,DOĞRU HABER,10892


In [41]:
type(rows_list[0][2])

numpy.float64

In [55]:
%matplotlib inline
rows_list_pd.plot(x="newspaper", y="circulation", kind="line")

[]

In [58]:
rows_list_pd["circulation"].plot()

<AxesSubplot:>